In [4]:
from newspaper import Article
from collectors.wikipedia import *
from collectors.google_news import *
from collectors.trends import *

import pandas as pd


### Get Trending celebrities

In [5]:
trendCollector = TrendCollector()
trending_celebs = await trendCollector.get_hottest_celebs(limit=100)
trending_celebs

['Josh Lucas',
 'Brianna Ruffalo',
 'Katy Perry',
 'Anthony Carrigan',
 'Leonardo DiCaprio',
 'Tobey Maguire',
 'Rashmika',
 'Wendy Williams',
 'Lindsay Lohan',
 'Jamie Lee Curtis',
 'Mike Shinoda',
 'Travis Kelce',
 'Bad Bunny',
 'Julie Bowen',
 'Jenna Bush Hager',
 'Billie Eilish',
 'James',
 'Lauren Sanchez',
 'Jeff Bezos',
 'Sydney Sweeney',
 'Rebecca Romijn',
 "Jerry O ' Connell",
 'Julianne Moore',
 'Liv',
 'Me',
 'Ricki Lake',
 'Sophia Bush',
 'Kim Zolciak',
 'Adam Sandler',
 'Kylie Jenner',
 'Mariska Hargitay',
 'Jayne Mansfield',
 'Wayne',
 'Coleen Rooney',
 'Johnny Depp',
 'Shah Rukh Khan',
 '##ult',
 'Sara Foster',
 'Joe Walsh',
 'Mandy Moore',
 'Taylor Goldsmith',
 'Danica Patrick',
 'Uorfi Javed',
 'Gwyneth Paltrow',
 'Brad Pitt',
 'Jennifer Aniston',
 'Star',
 'Tom Cruise',
 'William Shatner',
 'Lori Harvey',
 'Beyoncé',
 'Chris Hemsworth',
 'Apple Martin',
 'Mom Gwyneth Paltrow',
 'Madison LeCroy',
 'Rocker',
 'Anne',
 'William']

### Get wiki and profile infos via Wikipedia

In [6]:
from tqdm import tqdm

wikiCollector = WikipediaCollector()
profiles = []

for name in tqdm(trending_celebs, desc="Collecting celeb profiles", unit="celeb"):
    saved = wikiCollector.save_wiki(name)
    if not saved:
        continue
    
    profile = wikiCollector.profile(name)
    if profile:
        profiles.append(profile)

en.wikipedia.org (imageinfo) File:Ford v Ferrari PC 002 (48812302...
Josh Lucas (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Ford v Fe...
  infobox: <dict(11)> name, image, caption, birth_name, birth_date...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:J...
  pageid: 1147964
  parsetree: <str(29506)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: Josh Lucas
  wikibase: Q53651
  wikidata_url: https://www.wikidata.org/wiki/Q53651
  wikitext: <str(24676)> {{Short description|American actor (born ...
}
07/20/2025 05:08:32 PM - Successfully saved wiki of Josh Lucas
en.wikipedia.org (parse) Josh Lucas
en.wikipedia.org (imageinfo) File:Ford v Ferrari PC 002 (48812302...
Josh Lucas (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Ford v Fe...
  infobox: <dict(11)> name, image, caption, birth_name, birth_date...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:J...
  page

{{plainlist|
*|marriage|Jessica Ciencin Henriquez|2012|2014|end|=|divorced|
*|marriage|Brianna Ruffalo|2025}} {{marriage|Jessica Ciencin Henriquez|2012|2014|end|=|divorced}} * {{marriage|Brianna Ruffalo|2025}}


en.wikipedia.org (parse) Brianna Ruffalo
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
07/20/2025 05:08:33 PM - Failed to save wiki of Brianna Ruffalo: (https://en.wikipedia.org/w/api.php?action=parse&formatversion=2&contentmodel=text&disableeditsection=&disablelimitreport=&disabletoc=&prop=text|iwlinks|parsetree|wikitext|displaytitle|properties&redirects&page=Brianna%20Ruffalo)
en.wikipedia.org (imageinfo) File:KatyPerryWestminst111224 (81 of...
Katy Perry (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:KatyPerry...
  infobox: <dict(19)> name, image, alt, image_caption, birth_name,...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Category:K...
  pag

{{marriage|[[Russell Brand]]|2010|2012|end|=|divorced}}


/opt/anaconda3/envs/rag_llamaindex/lib/python3.13/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/anaconda3/envs/rag_llamaindex/lib/python3.13/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
07/20/2025 05:08:37 PM - Failed to save wiki of Anthony Carrigan: ("Anthony Carrigan" may refer to: 
Anthony Carrigan (actor)
Anthony Carrigan (academic))
07/20/2025 05:08:37 PM - Failed to save wiki of Anthony Carrigan: (Page id "anthony corrigan" does not match any pages. Try another id!)
en.wikipedia.org

...

en.wikipedia.org (parse) Chris Hemsworth
en.wikipedia.org (imageinfo) File:Chris Hemsworth by Gage Skidmor...
Chris Hemsworth (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Chris Hem...
  infobox: <dict(14)> name, honorific_suffix, image, caption, birt...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:C...
  pageid: 1221476
  parsetree: <str(108781)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Chris Hemsworth
  wikibase: Q54314
  wikidata_url: https://www.wikidata.org/wiki/Q54314
  wikitext: <str(93848)> {{Short description|Australian actor (bor...
}
07/20/2025 05:10:06 PM - Successfully saved wiki of Chris Hemsworth
en.wikipedia.org (parse) Chris Hemsworth
en.wikipedia.org (imageinfo) File:Chris Hemsworth by Gage Skidmor...
Chris Hemsworth (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Chris Hem...
  infobox: <dict(14)> name, honorific_suffix, image, caption, birt...
  iwlinks:

{{marriage|[[Elsa Pataky]]|2010}}


en.wikipedia.org (parse) Apple Martin
Apple Martin (en) data
{
  infobox: <dict(9)> name, birth_name, birth_date, birth_place, al...
  pageid: 1785299
  parsetree: <str(13677)> <root><template><title>Short description...
  requests: <list(1)> parse
  title: Apple Martin
  wikibase: Q47493349
  wikidata_url: https://www.wikidata.org/wiki/Q47493349
  wikitext: <str(10747)> {{Short description|English media persona...
}
07/20/2025 05:10:07 PM - Successfully saved wiki of Apple Martin
en.wikipedia.org (parse) Apple Martin
Apple Martin (en) data
{
  infobox: <dict(9)> name, birth_name, birth_date, birth_place, al...
  pageid: 1785299
  parsetree: <str(13677)> <root><template><title>Short description...
  requests: <list(1)> parse
  title: Apple Martin
  wikibase: Q47493349
  wikidata_url: https://www.wikidata.org/wiki/Q47493349
  wikitext: <str(10747)> {{Short description|English media persona...
}


None


en.wikipedia.org (parse) Mom Gwyneth Paltrow
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
07/20/2025 05:10:07 PM - Failed to save wiki of Mom Gwyneth Paltrow: (https://en.wikipedia.org/w/api.php?action=parse&formatversion=2&contentmodel=text&disableeditsection=&disablelimitreport=&disabletoc=&prop=text|iwlinks|parsetree|wikitext|displaytitle|properties&redirects&page=Mom%20Gwyneth%20Paltrow)
en.wikipedia.org (imageinfo) File:Southern Charm logo.jpg
Southern Charm (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Southern ...
  infobox: <dict(19)> image, genre, creator, starring, theme_music...
  pageid: 42076862
  parsetree: <str(46593)> <root><template><title>

In [ ]:
from dataclasses import asdict

profile_dicts = [asdict(p) for p in sorted(profiles, key = lambda x : x.name)]
celeb_profiles = pd.DataFrame(profile_dicts)
celeb_profiles.to_csv('../documents/profiles.csv')

### Collect Articles for Each “Hot Star”

In [13]:
from collectors.google_news import CelebrityGoogleNewsCollector
from newspaper import Article

news_collector = CelebrityGoogleNewsCollector()
news_collected = []

base_path = 'articles'


for p in profiles:
    news_list = await news_collector.get_news(p.name, max_results=10)
    base_md = {
        'celeb': p.name,
    }
    
    for news in news_list:
        await news_collector.save_article(
            url=news.url, 
            metadata={'source': news.publisher, **base_md})
    

07/20/2025 05:18:15 PM - Failed to save articles JSON Katy Perry(Article `download()` failed with 403 Client Error: Forbidden for url: https://www.sfchronicle.com/entertainment/article/katy-perry-sf-butterfly-20772665.php on URL https://www.sfchronicle.com/entertainment/article/katy-perry-sf-butterfly-20772665.php)
07/20/2025 05:18:21 PM - Failed to save articles JSON Katy Perry(Article `download()` failed with 404 Client Error: Not Found for url: https://www.foxnews.com/entertainment/katy-perry-left-dangling-midair-terrifying-concert-stage-malfunction on URL https://www.foxnews.com/entertainment/katy-perry-left-dangling-midair-terrifying-concert-stage-malfunction)
07/20/2025 05:18:32 PM - Failed to save articles JSON Leonardo DiCaprio(Article `download()` failed with 429 Client Error: Too Many Requests for url: https://www.realtor.com/news/celebrity-real-estate/leonardo-dicaprio-malibu-home-sale/ on URL https://www.realtor.com/news/celebrity-real-estate/leonardo-dicaprio-malibu-home-s